In [ ]:
!pip install -r requirements.txt
!pip install numpy==1.26.4 scikit-learn==1.3.2 --force-reinstall --no-cache-dir

In [ ]:
!pip install -U peft transformers accelerate huggingface_hub safetensors



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch
from huggingface_hub import login

login(token="hf_...")
model_id = "google/gemma-3-4b-pt"


# Путь к папке с сохранёнными файлами
checkpoint_dir = "./persistent_volume/lora_gemma/checkpoint-7500"  # заменишь на свою папку

# Загрузка конфигурации LoRA адаптера
peft_config = PeftConfig.from_pretrained(checkpoint_dir)

# Загружаем базовую модель (ту же, что и при обучении)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)


# Применяем LoRA адаптер
model = PeftModel.from_pretrained(base_model, checkpoint_dir)
model.eval()

# Загружаем токенайзер
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
tokenizer.pad_token = tokenizer.eos_token


In [2]:
# Пример исправленного кода:
prompt = "The secret to happiness is"

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# ❌ Без autocast, используем float32
with torch.no_grad():
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(output)



/opt/conda/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0709 10:44:32.915000 1738 site-packages/torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


The secret to happiness is to remain faithful to the real, and not to the ideal. Happiness is always a compensation, but only a very stupid or very bad man can be happy in the compensation itself. Happiness demands constant renunciation; the greater the renunciation the greater the happiness. He who loves only one woman can give himself wholly to her. With three or four friends he can only joke and never speak from the heart. And thus we gain happiness by renunciation. But this renunciation must not be a painful ren
